In [ ]:
import pandas as pd
import numpy as np
import os
import glob as gb
pd.options.display.max_columns = None
pd.set_option('display.max_rows',200)

In [ ]:
# ruta carpeta datos
ruta_datos = os.path.abspath("./../data/")
print(ruta_datos)
df_datos = pd.read_csv(ruta_datos+'/Hisminsa_2022/consolidados/df_consolidado_29122022.csv', encoding='UTF-8')
df_datos

In [ ]:
def curso_vida_joven(df_datos,**kwargs):
    act = kwargs['actividad']
    dx_empiesa_con = kwargs['dx_empiesa_con']
    dx2_empiesa_con = kwargs['dx2_empiesa_con']
    dx = kwargs['dx']
    lab = kwargs['lab']
    lab2 = kwargs['lab2']
    dx2 = kwargs['dx2']
    dx3 = kwargs['dx3']
    dx4 = kwargs['dx4']
    td = kwargs['td']
    td2 = kwargs['td2']
    td3 = kwargs['td3']
    enclab = kwargs['enclab']
    enclab2 = kwargs['enclab2']
    enclab_dif_de = kwargs['enclab_dif_de']
    cualquier_dx2 = kwargs['cualquier_dx2']
    gestantes = kwargs['gestantes']
    lab_isnull = kwargs['lab_isnull']
    inician = kwargs['inician']
    caso_tratado = kwargs['caso_tratado']
    
    df_joven = df_datos[df_datos['Anio_Actual_Paciente'].between(18,29)]

    # df_18_28 = df_datos[df_datos['Anio_Actual_Paciente'].between(18,28)]
    # df_29 = df_datos[(df_datos.Anio_Actual_Paciente == 29) & (df_datos.Mes_Actual_Paciente == 0) & (df_datos.Dia_Actual_Paciente == 0)]
    # df_joven = pd.concat([df_18_28,df_29], ignore_index=True)
    
    if not td:
        if dx and lab:
            df = df_joven[(df_joven.Codigo_Item.isin(dx)) & (df_joven.Valor_Lab.isin(lab))]
    elif (not enclab_dif_de) and (not enclab):
        if dx and lab:
            df = df_joven[(df_joven.Tipo_Diagnostico.isin(td)) & (df_joven.Codigo_Item.isin(dx)) & (df_joven.Valor_Lab.isin(lab))]
    else: # existe td, existe enclab, existe enclab_dif_de
        if not gestantes:
            if inician:
                if lab:
                    df_NoGestantes = df_joven[(~(df_joven.Valor_Lab.isin(enclab_dif_de))) & ((df_joven.Id_Condicion_Servicio.isin(['N','R'])))]
                    df = df_NoGestantes[(df_NoGestantes.Tipo_Diagnostico.isin(td)) & (df_NoGestantes.Codigo_Item.isin(dx)) & (df_NoGestantes.Valor_Lab.isin(lab))]
                    df = df.drop_duplicates(subset=['Id_Cita'])
                # else:
                #     #df_NoGestantes1 = df_joven[(df_joven.Tipo_Diagnostico.isin(td)) & (df_joven.Codigo_Item.isin(dx))]
                #     df_NoGestantes = df_joven[(~(df_joven.Valor_Lab.isin(enclab_dif_de))) & ((df_joven.Id_Condicion_Establecimiento.isin(['C']))) & ((df_joven.Id_Condicion_Servicio.isin(['C'])))]
                #     df = df_NoGestantes[(df_NoGestantes.Tipo_Diagnostico.isin(td)) & (df_NoGestantes.Codigo_Item.isin(dx))]
                #     #df = pd.merge(df_NoGestantes1,df_NoGestantes2, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'], how='inner')
                #     df = df.drop_duplicates(subset=['Id_Cita'])
                #     print('(ABO) no gestantes')
            # else:
            #     if lab:
            #         df_NoGestantes = df_joven[(~(df_joven.Valor_Lab.isin(enclab_dif_de))) & ((df_joven.Id_Condicion_Establecimiento.isin(['N','C','R']))) & ((df_joven.Id_Condicion_Servicio.isin(['N','R','C'])))]
            #         df = df_NoGestantes[(df_NoGestantes.Tipo_Diagnostico.isin(td)) & (df_NoGestantes.Codigo_Item.isin(dx)) & (df_NoGestantes.Valor_Lab.isin(lab))]
            #         df = df.drop_duplicates(subset=['Id_Cita'])
            #         print('(ABO) no gestantes')

            if caso_tratado:
                if lab:
                    df_NoGestantes = df_joven[(~(df_joven.Valor_Lab.isin(enclab_dif_de))) & ((df_joven.Id_Condicion_Servicio.isin(['C'])))]
                    df = df_NoGestantes[(df_NoGestantes.Tipo_Diagnostico.isin(td)) & (df_NoGestantes.Codigo_Item.isin(dx)) & (df_NoGestantes.Valor_Lab.isin(lab))]
                    df = df.drop_duplicates(subset=['Id_Cita'])
                    print('calculamos')
        else: #Gestantes
            if inician:
                if lab and enclab:
                    jovenes_gestantes = df_joven[(df_joven.Genero=='F')&((df_joven.Id_Otra_Condicion==1))&((df_joven.Valor_Lab.isin(enclab))|(df_joven.Valor_Lab.isnull())|(df_joven.Valor_Lab.isin(lab)))]
                    jovenes_labG = df_joven[(df_joven.Genero=='F')&(df_joven.Valor_Lab.isin(enclab))&(df_joven.Id_Otra_Condicion.isnull())]
                    jovenes_Gestantes_ini = pd.concat([jovenes_gestantes,jovenes_labG])
                    df = jovenes_Gestantes_ini[jovenes_Gestantes_ini.Tipo_Diagnostico.isin(td)&(jovenes_Gestantes_ini.Codigo_Item.isin(dx))&((jovenes_Gestantes_ini.Valor_Lab.isin(lab))|(jovenes_Gestantes_ini.Valor_Lab.isin(enclab)))]
                    
                    # df_Gestantes = df_joven[((df_joven.Genero.isin(['F']))) & ((df_joven.Valor_Lab.isin(enclab))) & ((df_joven.Id_Condicion_Servicio.isin(['N','R'])))]
                    # df = df_Gestantes[(df_Gestantes.Tipo_Diagnostico.isin(td)) & (df_Gestantes.Codigo_Item.isin(dx)) & (df_Gestantes.Valor_Lab.isin(lab))]
                    df = df.drop_duplicates(subset=['Id_Cita'])
                    print('gestantes inician')
                # else:
                #     df_Gestantes = df_joven[((df_joven.Genero.isin(['F']))) & ((df_joven.Valor_Lab.isin(enclab)))]
                #     df = df_Gestantes[(df_Gestantes.Tipo_Diagnostico.isin(td)) & (df_Gestantes.Codigo_Item.isin(dx))]
                #     df = df.drop_duplicates(subset=['Id_Cita'])
                #     print('(ABO) gestantes')
            # else:
            #     if lab:
            #         df_Gestantes = df_joven[((df_joven.Genero.isin(['F']))) & ((df_joven.Valor_Lab.isin(enclab)))]
            #         df = df_Gestantes[(df_Gestantes.Tipo_Diagnostico.isin(td)) & (df_Gestantes.Codigo_Item.isin(dx))]
            #         df = df.drop_duplicates(subset=['Id_Cita'])
            #         print('(ABO) gestantes----')
            if caso_tratado:
                df_Gestantes = df_joven[((df_joven.Genero.isin(['F']))) & ((df_joven.Valor_Lab.isin(enclab))) & ((df_joven.Id_Condicion_Establecimiento.isin(['C']))) & ((df_joven.Id_Condicion_Servicio.isin(['C'])))]
                df = df_Gestantes[(df_Gestantes.Tipo_Diagnostico.isin(td)) & (df_Gestantes.Codigo_Item.isin(dx)) & (df_Gestantes.Valor_Lab.isin(lab))]
                df = df.drop_duplicates(subset=['Id_Cita'])
        
        #df = pd.merge(df1,df, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
            #df = df[~(df.Valor_Lab.isin(dx_empiesa_con))]
    #df = df.groupby(['Anio','Mes']).size().reset_index(name='Nro')
    #df = df.pivot_table(index=['Anio'], columns='Mes', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()

    #df = df[df.Mes==10]

    df = df.groupby(['Anio','Mes'])['Id_Cita'].agg(['count']).reset_index()
    #df = df.rename({'count':'Activ','sum':'Insumo'}, axis=1)

    return df


In [ ]:
df_joven = df_datos[df_datos['Anio_Actual_Paciente'].between(18,29)]
df_joven

In [ ]:
#df_joven[df_joven.Id_Otra_Condicion==1]

In [ ]:
# df_18_28 = df_datos[df_datos['Anio_Actual_Paciente'].between(18,28)]
# df_18_28

In [ ]:
# df_joven.info()

In [ ]:
# df_29 = df_datos[(df_datos.Anio_Actual_Paciente == 29) & (df_datos.Mes_Actual_Paciente == 0) & (df_datos.Dia_Actual_Paciente == 0)]
# df_29


In [ ]:
# df_joven = pd.concat([df_18_28,df_29], ignore_index=True)
# df_joven

In [ ]:
# df_joven.info()

#probando

In [ ]:
# jovenes_gestantes_regla = df_joven[~(df_joven.Fecha_Ultima_Regla.isnull())]
# jovenes_gestantes_regla

In [ ]:
# obtener gestantes
jovenes_gestantes = df_joven[(df_joven.Genero=='F') & ((df_joven.Valor_Lab=='G') | (df_joven.Valor_Lab=='2'))]
jovenes_gestantes

In [ ]:
jovenes_gestantes[jovenes_gestantes.Id_Cita==794404012]

In [ ]:
# jovenes_labG = df_joven[(df_joven.Genero=='F')&(df_joven.Valor_Lab=='G') & (df_joven.Id_Otra_Condicion.isnull())]
# jovenes_labG = jovenes_labG[['Id_Cita','Codigo_Item','Valor_Lab','Id_Otra_Condicion']]
# jovenes_labG = jovenes_labG.rename({'Codigo_Item':'DXG','Valor_Lab':'LabG','Id_Otra_Condicion':'Gestante'}, axis=1)
# jovenes_labG
# jovenes_Gestantes_ini = pd.concat([jovenes_gestantes,jovenes_labG])
# jovenes_Gestantes_ini

In [ ]:
# jovenes_Gestantes_ini[jovenes_Gestantes_ini.Genero=='M']

In [ ]:
# df = pd.merge(jovenes_gestantes,jovenes_labG, on=['Id_Cita'], how='inner')
# df

In [ ]:
jovenes_inician = jovenes_gestantes[jovenes_gestantes.Tipo_Diagnostico.isin(['D'])&(jovenes_gestantes.Codigo_Item.isin(['D1310','D1330','D0120']))]
jovenes_inician.head(50)

In [ ]:
jovenes_inician[jovenes_inician.Id_Cita==764807442]


In [ ]:
# transponemos genero
df_dx1 = jovenes_inician.pivot(index=['Id_Cita', 'Anio', 'Mes', 'Dia', 'Fecha_Atencion', 'Id_Ups',
       'Id_Establecimiento', 'Id_Paciente', 'Id_Condicion_Establecimiento',
       'Id_Condicion_Servicio', 'Id_Personal', 'Id_Financiador',
       'Anio_Actual_Paciente', 'Mes_Actual_Paciente', 'Dia_Actual_Paciente',
       'Codigo_Item', 'Tipo_Diagnostico', 'Id_Correlativo',
        'Id_Pais', 'Id_Tipo_Documento','Numero_Documento',
        'Fecha_Nacimiento', 'Genero', 'Id_Etnia',
       'Id_Pais_Paciente'], columns=['Id_Correlativo_Lab'], values=['Valor_Lab']).reset_index()
df_dx1

In [ ]:
# transponemos genero
p = df_dx1.pivot(index=['Id_Cita', 'Anio', 'Mes', 'Dia', 'Fecha_Atencion', 'Id_Ups',
       'Id_Establecimiento', 'Id_Paciente', 'Id_Condicion_Establecimiento',
       'Id_Condicion_Servicio', 'Id_Personal', 'Id_Financiador',
       'Anio_Actual_Paciente', 'Mes_Actual_Paciente', 'Dia_Actual_Paciente',
       'Tipo_Diagnostico',
        'Id_Pais', 'Id_Tipo_Documento',
       'Numero_Documento', 'Fecha_Nacimiento', 'Genero', 'Id_Etnia',
       'Id_Pais_Paciente',('Valor_Lab',1),('Valor_Lab',2),('Valor_Lab',3)], columns=['Id_Correlativo'], values=['Codigo_Item']).reset_index()
p.head(50)

In [ ]:
p[p.Id_Cita==623658522]

In [ ]:
g=p.groupby(['Id_Cita', 'Anio', 'Mes', 'Dia', 'Fecha_Atencion', 'Id_Ups',
       'Id_Establecimiento', 'Id_Paciente', 'Id_Condicion_Establecimiento',
       'Id_Condicion_Servicio', 'Id_Personal', 'Id_Financiador',
       'Anio_Actual_Paciente', 'Mes_Actual_Paciente', 'Dia_Actual_Paciente',
       'Tipo_Diagnostico',
        'Id_Pais', 'Id_Tipo_Documento',
       'Numero_Documento', 'Fecha_Nacimiento', 'Genero', 'Id_Etnia',
       'Id_Pais_Paciente'])
g.first()

In [ ]:
df_dx1[df_dx1.Id_Cita==621562403]

In [ ]:
df1 = p[((p.Valor_Lab[1].isin(['G'])) | (p.Valor_Lab[2].isin(['G'])) | (p.Valor_Lab[3].isin(['G'])))]
df1

In [ ]:
df2 = p[((p.Valor_Lab[1].isin(['2'])) | (p.Valor_Lab[2].isin(['2'])) | (p.Valor_Lab[3].isin(['2'])))]
df2

In [ ]:
df = pd.merge(df1,df2, on=['Id_Cita', 'Anio', 'Mes', 'Dia', 'Fecha_Atencion', 'Id_Ups',
       'Id_Establecimiento', 'Id_Paciente', 'Id_Condicion_Establecimiento',
       'Id_Condicion_Servicio', 'Id_Personal', 'Id_Financiador',
       'Anio_Actual_Paciente', 'Mes_Actual_Paciente', 'Dia_Actual_Paciente',
       'Tipo_Diagnostico',
        'Id_Pais', 'Id_Tipo_Documento',
       'Numero_Documento', 'Fecha_Nacimiento', 'Genero', 'Id_Etnia',
       'Id_Pais_Paciente'], how='inner')
df

In [ ]:
df = df.drop_duplicates()

In [ ]:
df[df.Id_Cita==623658522]

In [ ]:
df2 = df.groupby(['Anio','Mes'])['Id_Cita'].agg(['count']).reset_index()
df2

In [ ]:
# jovenes_Lab1 = df_joven[df_joven.Valor_Lab=='1']
# jovenes_Lab1.info()

In [ ]:
# df = pd.merge(jovenes_dx,jovenes_Lab1, on=['Id_Cita'], how='inner')
# df.info()

In [ ]:
# dfX = pd.merge(df,jovenes_gestantes, on=['Id_Cita'], how='inner')
# dfX

# antes

In [ ]:
df_joven_g = df_joven[(df_joven.Genero.isin(['F'])) & (df_joven.Id_Otra_Condicion==1) & df_joven.Tipo_Diagnostico.isin(['D']) & (df_joven.Codigo_Item.isin(['Z012','D0150','D0120'])) & ((df_joven.Valor_Lab.isin(['G','1']))) ]
df_joven_g.head(100)

In [ ]:
df_joven_dx = df_joven[(df_joven.Genero.isin(['F'])) & df_joven.Tipo_Diagnostico.isin(['D']) & (df_joven.Codigo_Item.isin(['Z012','D0150','D0120'])) & ((df_joven.Valor_Lab.isin(['G','1'])))]
df_joven_dx.head(200)

In [ ]:
df = pd.merge(df_joven_g,df_joven_dx, on=['Id_Cita','Id_Paciente','Anio','Mes','Dia','Fecha_Atencion'], how='inner')
df.head(100)

In [ ]:
df.info()

In [ ]:
gestantesDX = pd.concat([df_joven_g,df_joven_dx])
gestantesDX.head(200)

In [ ]:
gestantesDX[gestantesDX.Id_Otra_Condicion.isnull()]

In [ ]:
df_joven_dxy =  gestantesDX[~((gestantesDX.Valor_Lab.isin(['1'])) & (gestantesDX.Id_Otra_Condicion.isnull()))]
df_joven_dxy

In [ ]:
# df = pd.merge(df_d_x,df_d_y, on=['Id_Cita','Id_Paciente','Anio','Mes','Dia','Fecha_Atencion','Id_Establecimiento','Id_Ups','Id_Condicion_Establecimiento','Id_Condicion_Servicio','Id_Personal','Id_Financiador',
#                                 'Anio_Actual_Paciente','Mes_Actual_Paciente','Dia_Actual_Paciente','Tipo_Diagnostico','Id_Pais',
#                                 'Id_Tipo_Documento','Numero_Documento','Fecha_Nacimiento','Genero','Id_Etnia','Id_Pais_Paciente'], how='inner')
# df.head(50)

In [ ]:
df_joven_dxy.info()

In [ ]:
df_joven_dxy.columns

In [ ]:
df_joven_dxy = df_joven_dxy.drop_duplicates(subset=['Id_Cita'])
df_joven_dxy

In [ ]:
# transponemos genero
df_dx1 = df_joven_dxy.pivot(index=['Id_Cita', 'Anio', 'Mes', 'Dia', 'Fecha_Atencion', 'Id_Ups',
       'Id_Establecimiento', 'Id_Paciente', 'Id_Condicion_Establecimiento',
       'Id_Condicion_Servicio', 'Id_Personal', 'Id_Financiador',
       'Anio_Actual_Paciente', 'Mes_Actual_Paciente', 'Dia_Actual_Paciente',
       'Codigo_Item', 'Tipo_Diagnostico', 'Id_Correlativo', 'Id_Otra_Condicion', 'Id_Centro_Poblado',
        'Fecha_Ultima_Regla','Id_Pais', 'Id_Tipo_Documento',
       'Numero_Documento', 'Fecha_Nacimiento', 'Genero', 'Id_Etnia',
       'Id_Pais_Paciente'], columns=['Id_Correlativo_Lab'], values=['Valor_Lab']).reset_index()
df_dx1

In [ ]:
# # transponemos genero
# df_dx2 = df_dx1.pivot(index=['Id_Cita', 'Anio', 'Mes', 'Dia', 'Fecha_Atencion', 'Id_Ups',
#        'Id_Establecimiento', 'Id_Paciente', 'Id_Condicion_Establecimiento',
#        'Id_Condicion_Servicio', 'Id_Personal', 'Id_Financiador',
#        'Anio_Actual_Paciente', 'Mes_Actual_Paciente', 'Dia_Actual_Paciente',
#        'Tipo_Diagnostico',
#         'Id_Pais', 'Id_Tipo_Documento',
#        'Numero_Documento', 'Fecha_Nacimiento', 'Genero', 'Id_Etnia',
#        'Id_Pais_Paciente',('Valor_Lab',1),('Valor_Lab',2),('Valor_Lab',3)], columns=['Id_Correlativo'], values=['Codigo_Item']).reset_index()
# df_dx2.head(50)

In [ ]:
df_dx1[df_dx1.Id_Cita.isin([638015512])]

In [ ]:
df_dx1.info()

In [ ]:
df_dx1.columns

In [ ]:
# df_dx1_na = df_dx1.dropna(how='all')
# df_dx1_na

In [ ]:
df1 = df_dx1[((df_dx1.Valor_Lab[1].isin(['G'])) | (df_dx1.Valor_Lab[2].isin(['G'])) | (df_dx1.Valor_Lab[3].isin(['G'])))]
df1

In [ ]:
df1[df1.Id_Cita.isin([638015512])]

In [ ]:
df2 = df_dx1[((df_dx1.Valor_Lab[1].isin(['1'])) | (df_dx1.Valor_Lab[2].isin(['1'])) | (df_dx1.Valor_Lab[3].isin(['1'])))]
df2

In [ ]:
df2[df2.Id_Cita.isin([638015512])]

In [ ]:
df = pd.merge(df1,df2, on=['Id_Paciente'], how='inner')
df

In [ ]:
#df[df.Id_Cita.isin([638015512])]
df[df.Id_Ups.isin([303201])]

In [ ]:
# df_dy= df[(df.Valor_Lab_y[1]=='1') | (df.Valor_Lab_y[2]=='1') | (df.Valor_Lab_y[2]=='1')]
# df_dy.head(200)

In [ ]:
df.info()

In [ ]:
# df_dy[df_dy.Id_Cita.isin([638015512])]

In [ ]:
# transponemos genero
df_dxx = df.pivot(index=['Id_Cita', 'Anio', 'Mes', 'Dia', 'Fecha_Atencion', 'Id_Ups',
       'Id_Establecimiento', 'Id_Paciente', 'Id_Condicion_Establecimiento',
       'Id_Condicion_Servicio', 'Id_Personal', 'Id_Financiador',
       'Anio_Actual_Paciente', 'Mes_Actual_Paciente', 'Dia_Actual_Paciente',
        'Tipo_Diagnostico', 
        'Id_Pais', 'Id_Tipo_Documento',
       'Numero_Documento', 'Fecha_Nacimiento', 'Genero', 'Id_Etnia',
       'Id_Pais_Paciente',('Valor_Lab_x',1),('Valor_Lab_x',2),('Valor_Lab_x',3),('Valor_Lab_y',1),('Valor_Lab_y',2),('Valor_Lab_y',3)], columns=['Id_Correlativo_x','Id_Correlativo_y'], values=['Codigo_Item_x','Codigo_Item_y']).reset_index()
df_dxx

In [ ]:
df_dxx[df_dxx.Id_Cita.isin([638015512])]

In [ ]:
df_dxxx = df_dxx[((df_dxx.Valor_Lab_y[1]=='1') | (df_dxx.Valor_Lab_y[2]=='1') | (df_dxx.Valor_Lab_y[3]=='1')) & ((df_dxx.Valor_Lab_x[1]=='G') | (df_dxx.Valor_Lab_x[2]=='G') | (df_dxx.Valor_Lab_x[3]=='G'))]
df_dxxx

In [ ]:
# df_dxx_na = df_dxx.dropna(how="all")
# df_dxx_na
df_dxxx.info()

In [ ]:
df_dxxx[df_dxxx.Id_Cita.isin([638015512])]

In [ ]:
# df = df_dx1.drop_duplicates(subset=[('Id_Cita','')])
# df

In [ ]:
# df_dx =  df_d[df_d.Tipo_Diagnostico.isin(['D']) &((df_d.Genero.isin(['F']))) & (df_d.Codigo_Item.isin(['Z012','D0150','D0120'])) & ((df_joven.Id_Condicion_Establecimiento.isin(['N']))) & ((df_joven.Id_Condicion_Servicio.isin(['N'])))]
# df_dx

In [ ]:
# df_dx.info()

In [ ]:
# agrupamos y contamos por genero
ji = jovenes_inician.groupby(["Id_Cita",'Id_Establecimiento','Id_Ups','Id_Personal','Id_Financiador','Anio','Mes','Dia','Numero_Documento','Anio_Actual_Paciente','Genero']).size().reset_index(name='Nro')
ji

In [ ]:
# transponemos genero
df_PAMS = ji.pivot_table(index=['Anio','Mes'], columns='Genero', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()
df_PAMS

In [ ]:
# # transponemos genero
# df_dx1 = ji.pivot(index=['Id_Cita',	'Anio',	'Mes',	'Dia',	'Fecha_Atencion',	'Id_Ups',	'Id_Establecimiento',	'Id_Paciente',	'Id_Condicion_Establecimiento',	'Id_Condicion_Servicio'	,'Id_Personal'	,'Id_Financiador'	,'Anio_Actual_Paciente'	,'Mes_Actual_Paciente',	'Dia_Actual_Paciente'	,'Tipo_Diagnostico','Id_Pais',	'Id_Tipo_Documento'	,'Numero_Documento',	'Fecha_Nacimiento',	'Genero'	,'Id_Etnia'	,'Id_Pais_Paciente'], columns=['Id_Correlativo'	,'Id_Correlativo_Lab'],values=['Valor_Lab','Codigo_Item']).reset_index()
# df_dx1.head(200)df_joven

In [ ]:
# df_dx1[df_dx1.Id_Cita==742085598]

In [ ]:
# df = pd.merge(df_dx,df_d, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='left')
# df

In [ ]:
# df = df.drop_duplicates(subset=['Id_Cita'])

In [ ]:
# df[df.Valor_Lab_x=='G']

In [ ]:
# df = df12[df12.Tipo_Diagnostico.isin(['D']) & (df12.Codigo_Item.isin(['Z012','D0150','D0120'])) & (df12.Valor_Lab.isin(['1']))]
# df

In [ ]:

#df = pd.merge(df12,df123, on=['Id_Cita','Anio','Mes','Dia','Id_Personal','Tipo_Diagnostico','Valor_Lab','Id_Correlativo_Lab','Genero'])
#df

In [ ]:
#df = df.drop_duplicates(subset=['Id_Cita','Anio','Mes','Dia','Id_Personal','Tipo_Diagnostico','Valor_Lab','Id_Correlativo_Lab','Genero'])
#df

In [ ]:
# df = df_dx1.drop_duplicates(subset=['Id_Cita'])
# df

In [ ]:
#df.isna().sum()

In [ ]:
df2 = jovenes_inician.groupby(['Anio','Mes'])['Id_Cita'].agg(['count']).reset_index()
df2

In [ ]:
df12 = df123[df123.Id_Correlativo_Lab.isin([1,2,3]) & ((df123.Genero =='F'))]
df12

In [ ]:
dff = pd.merge(df123,df12, on=['Id_Cita'])
dff

In [ ]:
df = df_joven[(df_joven.Codigo_Item.isin(['99401'])) & (df_joven.Valor_Lab.isin(['TA']))]
df

In [ ]:
a = curso_vida_joven(df_datos, actividad ='Plan de Atención Integral Elaborado', td=[], dx_empiesa_con=[], dx=['U169','C8002'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
a

a = curso_vida_joven(df_datos, actividad ='Plan de Atención Integral Ejecutado', td=[], dx_empiesa_con=[], dx=['U169','C8002'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
a

b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Indice de Masa Corporal Obesidad', td=['D'], dx_empiesa_con=[], dx=['E669'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Indice de Masa Corporal sobrepeso', td=['D'], dx_empiesa_con=[], dx=['E660'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Indice de Masa Corporal Normal', td=['D'], dx_empiesa_con=[], dx=['Z006'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Indice de Masa Corporal Delgadez', td=['D'], dx_empiesa_con=[], dx=['E440'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Indice de Masa Corporal Delgadez Severa', td=['D'], dx_empiesa_con=[], dx=['E43X'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Talla/Edad Talla Alta', td=['D'], dx_empiesa_con=[], dx=['E344'], cualquier_dx=[], lab=['TE'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Talla/Edad Talla Normal', td=['D'], dx_empiesa_con=[], dx=['Z006'], cualquier_dx=[], lab=['TE'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Talla/Edad Talla Baja', td=['D'], dx_empiesa_con=[], dx=['E45X'], cualquier_dx=[], lab=['TE'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=False,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Examen Odontológico no gestantes inician', td=['D'], dx_empiesa_con=[], dx=['Z012','D0150','D0120'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=['G'], gestantes=False, lab_isnull=False, inician=True,caso_tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Examen Odontológico no gestantes caso tratado', td=['D'], dx_empiesa_con=[], dx=['Z012','D0150','D0120'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=['G'], gestantes=False, lab_isnull=False,inician=False,caso_tratado=True)
b


b = curso_vida_joven(df_datos, actividad ='Instrucción de Higiene Oral Inician', td=['D'], dx_empiesa_con=[], dx=['D1330','U540'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=True,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Instrucción de Higiene Oral Caso Tratado', td=['D'], dx_empiesa_con=[], dx=['D1330','U540'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False, inician=False,tratado=True)
b
b = curso_vida_joven(df_datos, actividad ='Asesoria nutricional para el control de enfermedades dentales inician', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=True,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Asesoria nutricional para el control de enfermedades dentales caso tratado', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False, inician=False,tratado=True)
b
b = curso_vida_joven(df_datos, actividad ='Alta baśica odontológica (ABO) no gestantes', td=['D'], dx_empiesa_con=[], dx=['U510'], cualquier_dx=[], lab=['ABO'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=['G'], gestantes=False, lab_isnull=False, inician=False, tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Alta baśica odontológica (ABO) gestantes', td=['D'], dx_empiesa_con=[], dx=['U510'], cualquier_dx=[], lab=['ABO'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=True, lab_isnull=False, inician=False, tratado=False)

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Examen Odontológico gestantes inician', td=['D'], dx_empiesa_con=[], dx=['Z012','D0150','D0120'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=True, lab_isnull=False, inician=True,caso_tratado=False)
b


In [ ]:
b = curso_vida_joven(df_datos, actividad ='Examen Odontológico no gestantes caso tratado', td=['D'], dx_empiesa_con=[], dx=['Z012','D0150','D0120'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=['G'], gestantes=False, lab_isnull=False,inician=False,caso_tratado=True)
b


In [ ]:
b = curso_vida_joven(df_datos, actividad ='Instrucción de Higiene Oral Inician', td=['D'], dx_empiesa_con=[], dx=['D1330','U540'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=True,tratado=False)
b


In [ ]:
b = curso_vida_joven(df_datos, actividad ='Instrucción de Higiene Oral Caso Tratado', td=['D'], dx_empiesa_con=[], dx=['D1330','U540'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False, inician=False,tratado=True)
b
b = curso_vida_joven(df_datos, actividad ='Asesoria nutricional para el control de enfermedades dentales inician', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False,inician=True,tratado=False)
b
b = curso_vida_joven(df_datos, actividad ='Asesoria nutricional para el control de enfermedades dentales caso tratado', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_dif_de=[], gestantes=False, lab_isnull=False, inician=False,tratado=True)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b

In [ ]:
b = curso_vida_joven(df_datos, actividad ='Consejería Integral Finalizada', td=['D'], dx_empiesa_con=[], dx=['99401'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
b